In [6]:
import numpy as np
import os
import pandas as pd
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import warnings
import math


In [7]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\src\\data_preprocessing\\tsdm\\')
mandal_boundaries = gpd.read_file('Mandal_Boundary.shp')
mandal_boundaries = mandal_boundaries.reset_index()

os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\src\\data_preprocessing\\tsdm\\')
district_boundaries = gpd.read_file('District_Boundary.shp')
district_boundaries = district_boundaries.reset_index()

os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\src\\data_preprocessing\\base_geojson')
shape_telangana = gpd.read_file('TL_state_shapefile_for_clip.geojson')
shape_telangana = shape_telangana.explode()

warnings.filterwarnings('ignore')


In [8]:
#https://towardsdatascience.com/timeseries-data-munging-lagging-variables-that-are-distributed-across-multiple-groups-86e0a038460c
def lag_by_group(key, value_df):
    df = value_df.assign(group = key) # this pandas method returns a copy of the df, with group columns assigned the key value
    return (df.sort_values(by=["ModifiedDateTime"], ascending=True).set_index(["ModifiedDateTime"]).shift(12)) # the parenthesis allow you to chain methods and avoid intermediate variable assignment

In [9]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Data_csv\\results_csv')
df = pd.read_csv('data_tiffs.csv', index_col = 0)

In [10]:
df.columns

Index(['index', 'Mandal_Nam', 'Dist_Name', 'geometry', 'ModifiedDateTime',
       'Year', 'Month', 'Season', 'NO2', 'PM2.5', 'SSM_mean', 'SSM_median',
       'SSM_min', 'SSM_max', 'SSM_percentile_90', 'PRE_mean', 'PRE_median',
       'PRE_min', 'PRE_max', 'PRE_percentile_90', 'PRE_sum', 'ST_mean',
       'ST_median', 'ST_min', 'ST_max', 'ST_percentile_90', 'RWI_mean',
       'RWI_median', 'RWI_min', 'RWI_max', 'RWI_percentile_90', 'BA_mean',
       'BA_median', 'BA_sum', 'Crops', 'Crops_Prop', 'Trees', 'Trees_Prop',
       'Built Area', 'Built_Area_Prop', 'time_to_rur_median', 'PowP_score',
       'Soil Type', 'Area', 'POP_mean', 'POP_median'],
      dtype='object')

# Crop types

In [11]:
os.chdir('C:\\Users\\Jesse\\OneDrive\\Documenten\\Data_thesis\\crop_area_data')

filenames = os.listdir('C:\\Users\\Jesse\\OneDrive\\Documenten\\Data_thesis\\crop_area_data')
frames = []

for j in filenames:
    df1 = pd.read_csv(j)
    if j == 'mahabubabad_kharif_2016-17.csv':
        df1.columns = ['year', 'season', 'districtName', 'crop', 'mandalName', 'normalAreaSown', 'actualAreaSown']
    else:
        df1.columns = ['year', 'season', 'districtName', 'mandalName', 'crop','normalAreaSown', 'actualAreaSown']
   
    frames.append(df1)  
        
result = pd.concat(frames)



In [12]:
result['season'] = result['season'].replace('Karif', 'Kharif')

In [13]:
result['year'] = result['year'].replace(2016, '2016-2017')
result['year'] = result['year'].replace(2017, '2017-2018')
result['year'] = result['year'].replace(2018, '2018-2019')
result['year'] = result['year'].replace('2017-18', '2017-2018')
result['year'] = result['year'].replace('2018-19', '2018-2019')
result['year'] = result['year'].replace('2016-17', '2016-2017')
result['year'] = result['year'].replace('2019-2019', '2018-2019')

In [14]:
result['year'].unique()

array(['2016-2017', '2017-2018', '2018-2019', '2014-2015', '2015-2016'],
      dtype=object)

In [15]:
#Rewite Year column
result['Year'] = 0
result['year'] = result['year'].astype(str)
for i in range(0, len(result)):
    if result['season'].iloc[i] == 'Rabi':
        result['Year'].iloc[i] = result['year'].iloc[i][5:9]
    elif result['season'].iloc[i] == 'Kharif':
        result['Year'].iloc[i] = result['year'].iloc[i][0:4]

del result['year']

In [16]:
result.columns = ['Season', 'Dist_Name', 'Mandal_Nam', 'Crop Type', 'NormalAreaSown', 'ActualAreaSown', 'Year']

In [17]:
foods = ['Cashewnut', 'Tobacco', 'Turmeric', 'Acid Lime', 'Ajwam (Vamu)', 'Amala', 'Anumulu', 'Apples', 'Ash Gourd', 'Bajra', 'Banana', 'Batavia', 'Bengal', 'Bhendi', 'Bitter Guard', 'Black', 'Bodakakara', 'Bottle Guard', 
'Brinjal', 'Cabbage', 'Cauliflower', 'Capsicum', 'Carrot', 'Castor', 'Chikkudu', 'Chillies', 'Coccinea', 'Coconut', 'Coriander', 'Cotton', 'Cow', 'Cucumber', 'Custard Apple', 'Danimma', 'Donda', 'Drum Stick', 'Eucalyptus', 
'Flowers', 'Jowar', 'Fruit', 'Ginger', 'Gokara', 'Grapes', 'Green', 'Groundnut', 'Guava', 'Horse', 'Horticulture', 'Jowar', 'Jasmine', 'Korra', 'Kothemmeera', 'Maize', 'Mango', 'Mulberry', 'Mustard', 'Onions', 'Other', 'Orange',
'Paddy', 'Palm Oil', 'Papaya', 'Peas', 'Plantain', 'Pomegranate', 'Potato', 'Ragi', 'Red', 'Rice', 'Ridge Gourd', 'Rose', 'Safflower', 'Sesamum', 'Soybean', 'Sugarcane', 'Sunflower', 'Tobacco', 'Tomatoes', 'Wheat', 'Total', 'Vegetables']
#colors = ['Black', 'Cow', 'Red', 'Green', 'Bengal', 'Horse']

In [18]:
#Delete a few uninteresting columns
result = result[(result['Crop Type'] != 'Total') & (result['ActualAreaSown']!=0) & (result['Crop Type'] != 'Gross Area Sown') & (result['Crop Type'] != 'Gross Area') & (result['Crop Type'] != 'Gross areas ') & (result['Crop Type'] != 'Grand Total') & (result['Crop Type'] != 'Grand Total') ]
result = result.dropna(subset=['ActualAreaSown'])

In [19]:
result['ActualAreaSown'] = result['ActualAreaSown']*0.01

In [20]:
#Find most similar districts
max_score = 0
max_district = ''
result['Matched District'] = 'None'

for line in range(0, len(result)):
    for district in range(0, len(mandal_boundaries['Dist_Name'].unique())):
        score = fuzz.token_set_ratio(mandal_boundaries['Dist_Name'].unique()[district], result['Dist_Name'].iloc[line])
        if score>max_score:
            max_district = mandal_boundaries['Dist_Name'].unique()[district]
            max_score = score

    result['Matched District'].iloc[line] = max_district
    max_score = 0
    max_district = ''
        

In [21]:
#Change wrong districts by hand
for i in range(0, len(result)):
    if result['Dist_Name'].iloc[i] == 'Medchal':
        result['Matched District'].iloc[i] = 'Medchal_Malkajgiri'
    elif result['Dist_Name'].iloc[i] == 'Warangal (Urban)':
        result['Matched District'].iloc[i] = 'Warangal_Urban'

In [22]:
#Find most similar mandal within the selected district
max_score = 0
max_mandal = ''
result['Matched Mandal'] = 'None'

for line in range(0, len(result)):
    district = mandal_boundaries[mandal_boundaries['Dist_Name'] == result['Matched District'].iloc[line]]
    for mandal in range(0, len(district)):
        score = fuzz.token_set_ratio(district['Mandal_Nam'].iloc[mandal], result['Mandal_Nam'].iloc[line])
        if score>max_score:
            max_mandal = district['Mandal_Nam'].iloc[mandal]
            max_score = score

    result['Matched Mandal'].iloc[line] = max_mandal
    max_score = 0
    max_mandal = ''
        


In [23]:
#Find the most similar crop type
max_score = 0
max_crop = ''
result['Matched Crop'] = 'None'

for line in range(0, len(result)):
    for crop in range(0, len(foods)):
        score = fuzz.token_set_ratio(foods[crop], result['Crop Type'].iloc[line])
        if score>max_score:
            max_crop = foods[crop]
            max_score = score

    result['Matched Crop'].iloc[line] = max_crop
    max_score = 0
    max_crop = ''

In [24]:
result['Matched Crop'] = result['Matched Crop'].replace('Rice', 'Paddy')
result_test = result[['Season', 'Dist_Name', 'Mandal_Nam', 'Year', 'ActualAreaSown', 'Matched Mandal', 'Matched District', 'Matched Crop']]


In [25]:
pivoted = pd.pivot_table(result_test, values='ActualAreaSown', index=['Season', 'Year', 'Dist_Name', 'Mandal_Nam', 'Matched Mandal', 'Matched District'],
                    columns=['Matched Crop'], aggfunc='mean', fill_value=0).reset_index()

In [26]:
pivoted.columns = ['Season', 'Year', 'Actual Dist', 'Actual Mandal', 'Mandal_Nam', 'Dist_Name', 'Acid Lime', 'Ajwam (Vamu)', 'Amala', 'Anumulu',
       'Apples', 'Ash Gourd', 'Bajra', 'Banana', 'Batavia', 'Bengal', 'Bhendi',
       'Bitter Guard', 'Black', 'Bodakakara', 'Bottle Guard', 'Brinjal',
       'Cabbage', 'Capsicum', 'Carrot', 'Cashewnut', 'Castor', 'Cauliflower',
       'Chikkudu', 'Chillies', 'Coccinea', 'Coriander', 'Cotton', 'Cow',
       'Cucumber', 'Custard Apple', 'Danimma', 'Donda', 'Drum Stick',
       'Flowers', 'Fruit', 'Ginger', 'Gokara', 'Grapes', 'Green', 'Groundnut',
       'Guava', 'Horse', 'Horticulture', 'Jasmine', 'Jowar', 'Korra',
       'Kothemmeera', 'Maize', 'Mango', 'Mulberry', 'Mustard', 'Onions',
       'Orange', 'Other', 'Paddy', 'Palm Oil', 'Papaya', 'Peas', 'Plantain',
       'Pomegranate', 'Potato', 'Ragi', 'Red', 'Ridge Gourd', 'Rose',
       'Safflower', 'Sesamum', 'Soybean', 'Sugarcane', 'Sunflower', 'Tobacco',
       'Tomatoes', 'Total', 'Turmeric', 'Vegetables', 'Wheat']

In [27]:
for a in range(0, len(pivoted)):
    #Adilabad
    if (pivoted['Dist_Name'].iloc[a] == 'Adilabad') & (pivoted['Actual Mandal'].iloc[a] == 'Adilabad U'):
        pivoted['Mandal_Nam'].iloc[a] = 'Adilabad Urban'
    #Bhadradri Kothagudem
    elif (pivoted['Dist_Name'].iloc[a] == 'Bhadradri Kothagudem') & (pivoted['Actual Mandal'].iloc[a] == 'Onions'):
        pivoted['Mandal_Nam'].iloc[a] = 'Unknown'
    #Jagtial
    elif (pivoted['Dist_Name'].iloc[a] == 'Jagtial') & ((pivoted['Actual Mandal'].iloc[a] == 'Jagtial R') | (pivoted['Actual Mandal'].iloc[a] == 'Jagtial R ')):
        pivoted['Mandal_Nam'].iloc[a] = 'Jagityal_Rural'
    #Jangoan
    elif (pivoted['Dist_Name'].iloc[a] == 'Jangoan') & ((pivoted['Actual Mandal'].iloc[a] == 'Lingal Ghanpur') | (pivoted['Actual Mandal'].iloc[a] == 'Lingala Ghanpur')):
        pivoted['Mandal_Nam'].iloc[a] = 'Lingalaghanpur'
    elif (pivoted['Dist_Name'].iloc[a] == 'Jangoan') & ((pivoted['Actual Mandal'].iloc[a] == 'Gundala')):
        pivoted['Mandal_Nam'].iloc[a] = 'Unknown'
    #Jayashankar Bhupalpally
    elif  (pivoted['Dist_Name'].iloc[a] == 'Jayashankar Bhupalpally') & ((pivoted['Actual Mandal'].iloc[a] == 'Eturnagaram') |  (pivoted['Actual Mandal'].iloc[a] == 'Eturunagaram') | (pivoted['Actual Mandal'].iloc[a] == 'Kannaigudem') | (pivoted['Actual Mandal'].iloc[a] == 'Mahamutharam') |  (pivoted['Actual Mandal'].iloc[a] == 'Mahamuttaram') | (pivoted['Actual Mandal'].iloc[a] == 'Tadvai') | (pivoted['Actual Mandal'].iloc[a] == 'VENKATAPURAM')|  (pivoted['Actual Mandal'].iloc[a] == 'Venkatapur') | (pivoted['Actual Mandal'].iloc[a] == 'Venkatapur ') | (pivoted['Actual Mandal'].iloc[a] == 'Venkatapuram')|  (pivoted['Actual Mandal'].iloc[a] == 'Venkatapuram ') | (pivoted['Actual Mandal'].iloc[a] == 'Mangapet') | (pivoted['Actual Mandal'].iloc[a] == 'Wajedu') | (pivoted['Actual Mandal'].iloc[a] == 'Govindaraopet')):
        pivoted['Mandal_Nam'].iloc[a] = 'Unknown'
    #Karimnagar
    elif (pivoted['Dist_Name'].iloc[a] == 'Karimnagar') & ((pivoted['Actual Mandal'].iloc[a] == 'Karimnagar (Rural)') | (pivoted['Actual Mandal'].iloc[a] == 'Karimnagar Rural ') | (pivoted['Actual Mandal'].iloc[a] == 'Karimnagar Rural')):
        pivoted['Mandal_Nam'].iloc[a] = 'Karimnagar_Rural'
    #Khammam 
    elif (pivoted['Dist_Name'].iloc[a] == 'Khammam') & ((pivoted['Actual Mandal'].iloc[a] == 'Khammam (U)') | (pivoted['Actual Mandal'].iloc[a] == 'Khammam (u)')):
        pivoted['Mandal_Nam'].iloc[a] = 'Khammam_Urban'
    #Mahabubnagar
    elif (pivoted['Dist_Name'].iloc[a] == 'Mahabubnagar') & ((pivoted['Actual Mandal'].iloc[a] == 'Maddur') | (pivoted['Actual Mandal'].iloc[a] == 'Makthal') | (pivoted['Actual Mandal'].iloc[a] == 'Marikal') | (pivoted['Actual Mandal'].iloc[a] == 'Maganoor') | (pivoted['Actual Mandal'].iloc[a] == 'Utkoor')| (pivoted['Actual Mandal'].iloc[a] == 'Utkur') | (pivoted['Actual Mandal'].iloc[a] == 'Damaragidda')| (pivoted['Actual Mandal'].iloc[a] == 'Damargidda') | (pivoted['Actual Mandal'].iloc[a] == 'Dhanwada')| (pivoted['Actual Mandal'].iloc[a] == 'Narva')| (pivoted['Actual Mandal'].iloc[a] == 'Kosgi') | (pivoted['Actual Mandal'].iloc[a] == 'Krishna')| (pivoted['Actual Mandal'].iloc[a] == 'Narayanpet') | (pivoted['Actual Mandal'].iloc[a] == 'Narwa')):
        pivoted['Mandal_Nam'].iloc[a] = 'Unknown'
    elif (pivoted['Dist_Name'].iloc[a] == 'Mahabubnagar') & ((pivoted['Actual Mandal'].iloc[a] == 'Mahabubnagar ( r )') | (pivoted['Actual Mandal'].iloc[a] == 'Mahabubnagar (r)') | (pivoted['Actual Mandal'].iloc[a] == 'Mahabubnagar(r)')):
        pivoted['Mandal_Nam'].iloc[a] = 'Mahabubnagar_Rural'
    #Medak
    elif (pivoted['Dist_Name'].iloc[a] == 'Medak') & (pivoted['Actual Mandal'].iloc[a] == 'Shankarampet (R)'):
        pivoted['Mandal_Nam'].iloc[a] = 'Shankarampet_R'
    #Nalgonda
    elif (pivoted['Dist_Name'].iloc[a] == 'Nalgonda') & ((pivoted['Actual Mandal'].iloc[a] == 'P a pally') |  (pivoted['Actual Mandal'].iloc[a] == 'P.A.Pally') ):
        pivoted['Mandal_Nam'].iloc[a] = 'Peda Adisharla Palli'
    #Nirmal
    elif (pivoted['Dist_Name'].iloc[a] == 'Nirmal') & (pivoted['Actual Mandal'].iloc[a] == 'Nirmal Rural'):
        pivoted['Mandal_Nam'].iloc[a] = 'Nirmal Rural'
    #Nizamabad
    elif (pivoted['Dist_Name'].iloc[a] == 'Nizamabad') & (pivoted['Actual Mandal'].iloc[a] == 'Yeda Palle'):
        pivoted['Mandal_Nam'].iloc[a] = 'Yedapally'
    #Peddapalli
    elif (pivoted['Dist_Name'].iloc[a] == 'Peddapalli') & ((pivoted['Actual Mandal'].iloc[a] == 'Mutharam manthani')|(pivoted['Actual Mandal'].iloc[a] == 'Mutharam(manthani)')):
        pivoted['Mandal_Nam'].iloc[a] = 'Mutharam_Manthani'
    #Rajanna Sircilla
    elif (pivoted['Dist_Name'].iloc[a] == 'Rajanna Sircilla') & ((pivoted['Actual Mandal'].iloc[a] == 'Vemulawada (R)')|(pivoted['Actual Mandal'].iloc[a] == 'Vemulawada (rural)')|(pivoted['Actual Mandal'].iloc[a] == 'Vemulawada( rural)')):
        pivoted['Mandal_Nam'].iloc[a] = 'Vemulawada Rural'
    #Rangareddy
    elif (pivoted['Dist_Name'].iloc[a] == 'Rangareddy') & ((pivoted['Actual Mandal'].iloc[a] == 'Qutbullahpur')|(pivoted['Actual Mandal'].iloc[a] == 'Quthbullapur')|(pivoted['Actual Mandal'].iloc[a] == 'Yalal')|(pivoted['Actual Mandal'].iloc[a] == 'Malkajgiri')|(pivoted['Actual Mandal'].iloc[a] == 'Pudur')|(pivoted['Actual Mandal'].iloc[a] == 'Kulkacherla')|(pivoted['Actual Mandal'].iloc[a] == 'Pargi')|(pivoted['Actual Mandal'].iloc[a] == 'Gandeed')|(pivoted['Actual Mandal'].iloc[a] == 'Mominpet')|(pivoted['Actual Mandal'].iloc[a] == 'Nawabpet')|(pivoted['Actual Mandal'].iloc[a] == 'Tandur')|(pivoted['Actual Mandal'].iloc[a] == 'Keesara')|(pivoted['Actual Mandal'].iloc[a] == 'Shameerpet')|(pivoted['Actual Mandal'].iloc[a] == 'Shamirpet')|(pivoted['Actual Mandal'].iloc[a] == 'Dharur')|(pivoted['Actual Mandal'].iloc[a] == 'Peddemul')|(pivoted['Actual Mandal'].iloc[a] == 'Uppal')|(pivoted['Actual Mandal'].iloc[a] == 'Ghatkesar')|(pivoted['Actual Mandal'].iloc[a] == 'Vikarabad')|(pivoted['Actual Mandal'].iloc[a] == 'Doma')|(pivoted['Actual Mandal'].iloc[a] == 'Parigi')|(pivoted['Actual Mandal'].iloc[a] == 'Basheerabad')|(pivoted['Actual Mandal'].iloc[a] == 'Marpalle')|(pivoted['Actual Mandal'].iloc[a] == 'Marpally')|(pivoted['Actual Mandal'].iloc[a] == 'Basheerabad')|(pivoted['Actual Mandal'].iloc[a] == 'Bantaram')|(pivoted['Actual Mandal'].iloc[a] == 'Kulkacharla')):
        pivoted['Mandal_Nam'].iloc[a] = 'Unknown'
    #Sangareddy
    elif (pivoted['Dist_Name'].iloc[a] == 'Sangareddy') & ((pivoted['Actual Mandal'].iloc[a] == 'Jagadevpur')|(pivoted['Actual Mandal'].iloc[a] == 'Chinnakodur')|(pivoted['Actual Mandal'].iloc[a] == 'Medak')|(pivoted['Actual Mandal'].iloc[a] == 'Toopran')|(pivoted['Actual Mandal'].iloc[a] == 'Alladurg')|(pivoted['Actual Mandal'].iloc[a] == 'Kulcharam')|(pivoted['Actual Mandal'].iloc[a] == 'Tekmal')|(pivoted['Actual Mandal'].iloc[a] == 'Narsapur')|(pivoted['Actual Mandal'].iloc[a] == 'Yeldurthy')|(pivoted['Actual Mandal'].iloc[a] == 'Nangnoor')|(pivoted['Actual Mandal'].iloc[a] == 'Kowdipally')|(pivoted['Actual Mandal'].iloc[a] == 'Toguta')|(pivoted['Actual Mandal'].iloc[a] == 'Mulugu')|(pivoted['Actual Mandal'].iloc[a] == 'Gajwel')|(pivoted['Actual Mandal'].iloc[a] == 'Wargal')|(pivoted['Actual Mandal'].iloc[a] == 'Dubbak')|(pivoted['Actual Mandal'].iloc[a] == 'Mirdoddi')|(pivoted['Actual Mandal'].iloc[a] == 'Shankarampet-A')|(pivoted['Actual Mandal'].iloc[a] == 'Shankarampet-R')|(pivoted['Actual Mandal'].iloc[a] == 'Doma')|(pivoted['Actual Mandal'].iloc[a] == 'Jogipet')|(pivoted['Actual Mandal'].iloc[a] == 'Papannapet')|(pivoted['Actual Mandal'].iloc[a] == 'Ramayampet')|(pivoted['Actual Mandal'].iloc[a] == 'Shivampet')|(pivoted['Actual Mandal'].iloc[a] == 'Siddipet')|(pivoted['Actual Mandal'].iloc[a] == 'Velimela')|(pivoted['Actual Mandal'].iloc[a] == 'Doulthabad') |(pivoted['Actual Mandal'].iloc[a] == 'Chegunta')):
        pivoted['Mandal_Nam'].iloc[a] = 'Unknown'
    #Warangal_Urban
    elif (pivoted['Dist_Name'].iloc[a] == 'Warangal_Urban') & ((pivoted['Actual Mandal'].iloc[a] == 'Warangal')):
        pivoted['Mandal_Nam'].iloc[a] = 'Warangal'
    elif (pivoted['Dist_Name'].iloc[a] == 'Warangal_Urban') & ((pivoted['Actual Mandal'].iloc[a] == 'Killa-warangal')):
        pivoted['Mandal_Nam'].iloc[a] = 'Khila Warangal'



In [28]:
matched = pd.merge(pivoted, mandal_boundaries[['index', 'Mandal_Nam', 'Dist_Name']], how = 'inner', on=['Mandal_Nam', 'Dist_Name'])

In [29]:
matched_pivots = pd.merge(pivoted, mandal_boundaries[['index', 'Mandal_Nam', 'Dist_Name']], how = 'inner', on=['Mandal_Nam', 'Dist_Name'])
all_matched_pivots = pd.merge(pivoted, mandal_boundaries[['index', 'Mandal_Nam', 'Dist_Name']], how = 'left', on=['Mandal_Nam', 'Dist_Name'])
all_matched_mandals = pd.merge(pivoted, mandal_boundaries[['index', 'Mandal_Nam', 'Dist_Name']], how = 'right', on=['Mandal_Nam', 'Dist_Name'])

In [30]:
unknowns = all_matched_pivots[all_matched_pivots['Mandal_Nam']=='Unknown']
missing_mandals = all_matched_mandals[(all_matched_mandals['Actual Dist'].isnull()) &  (all_matched_mandals['Dist_Name']!='Hyderabad')]

In [31]:
#Match unknowns from district with the missing mandals
max_score = 0
max_crop = ''
unknowns['Matching Dist'] = 0

for line in range(0, len(unknowns)):
    for mandals in range(0, len(missing_mandals)):
        score = fuzz.token_set_ratio(missing_mandals['Mandal_Nam'].iloc[mandals], unknowns['Actual Mandal'].iloc[line])
        if score>max_score:
            max_crop = missing_mandals['Mandal_Nam'].iloc[mandals]
            dist = missing_mandals['Dist_Name'].iloc[mandals]
            max_score = score

    unknowns['Mandal_Nam'].iloc[line] = max_crop
    unknowns['Matching Dist'].iloc[line] = dist
    max_score = 0
    max_crop = ''

In [32]:
for i in range(0, len(unknowns)):
    if (unknowns['Dist_Name'].iloc[i] == 'Bhadradri Kothagudem') & (unknowns['Actual Mandal'].iloc[i] == 'Onions'):
        unknowns['Mandal_Nam'].iloc[i]='Unknown'
    elif (unknowns['Dist_Name'].iloc[i] == 'Jayashankar Bhupalpally') & ((unknowns['Actual Mandal'].iloc[i] == 'Mahamutharam')|(unknowns['Actual Mandal'].iloc[i] == 'Mahamuttaram')|(unknowns['Actual Mandal'].iloc[i] == 'Wajedu')):
        unknowns['Mandal_Nam'].iloc[i]='Unknown'
    elif (unknowns['Dist_Name'].iloc[i] == 'Rangareddy') & ((unknowns['Actual Mandal'].iloc[i] == 'Bantaram')|(unknowns['Actual Mandal'].iloc[i] == 'Basheerabad')|(unknowns['Actual Mandal'].iloc[i] == 'Dharur')|(unknowns['Actual Mandal'].iloc[i] == 'Doma')|(unknowns['Actual Mandal'].iloc[i] == 'Gandeed')|(unknowns['Actual Mandal'].iloc[i] == 'Kulkacharla')|(unknowns['Actual Mandal'].iloc[i] == 'Kulkacherla')|(unknowns['Actual Mandal'].iloc[i] == 'Mominpet')|(unknowns['Actual Mandal'].iloc[i] == 'Nawabpet')|(unknowns['Actual Mandal'].iloc[i] == 'Pargi')|(unknowns['Actual Mandal'].iloc[i] == 'Parigi')|(unknowns['Actual Mandal'].iloc[i] == 'Peddemul')|(unknowns['Actual Mandal'].iloc[i] == 'Pudur')|(unknowns['Actual Mandal'].iloc[i] == 'Tandur')|(unknowns['Actual Mandal'].iloc[i] == 'Vikarabad')|(unknowns['Actual Mandal'].iloc[i] == 'Yalal')):
        unknowns['Mandal_Nam'].iloc[i]='Unknown'
    elif (unknowns['Dist_Name'].iloc[i] == 'Rangareddy') & ((unknowns['Actual Mandal'].iloc[i] == 'Marpalle')):
        unknowns['Mandal_Nam'].iloc[i]='Unknown'
    elif  (unknowns['Dist_Name'].iloc[i] == 'Sangareddy') &  (unknowns['Actual Mandal'].iloc[i] in ['Alladurg','Chegunta','Chinnakodur','Doulthabad','Dubbak','Gajwel','Jagadevpur','Jogipet','Kowdipally','Kulcharam','Medak','Mirdoddi','Nangnoor','Narsapur','Papannapet','Ramayampet','Shankarampet-A','Shankarampet-R','Shivampet','Siddipet','Tekmal','Toguta','Toopran','Velimela','Wargal','Yeldurthy']):
        unknowns['Mandal_Nam'].iloc[i]='Unknown'


In [33]:
new_adds = unknowns[unknowns['Mandal_Nam']!='Unknown']
new_adds['Dist_Name'] = new_adds['Matching Dist']
del new_adds['Matching Dist']
del new_adds['index']
new_unknowns = unknowns[unknowns['Mandal_Nam']=='Unknown']
del new_unknowns['Matching Dist']
del new_unknowns['index']

In [34]:
linked = pd.merge(new_adds, mandal_boundaries[['Dist_Name', 'Mandal_Nam', 'index']], on =['Dist_Name', 'Mandal_Nam'], how='left')
matched_pivots = matched_pivots.append(linked)

In [35]:
#Match unknowns with all possible mandals
max_score = 0
max_crop = ''
new_unknowns['Matching Dist'] = 0


for line in range(0, len(new_unknowns)):
    for mandals in range(0, len(mandal_boundaries)):
        score = fuzz.token_set_ratio(mandal_boundaries['Mandal_Nam'].iloc[mandals], new_unknowns['Actual Mandal'].iloc[line])
        if score>max_score:
            max_crop = mandal_boundaries['Mandal_Nam'].iloc[mandals]
            dist = mandal_boundaries['Dist_Name'].iloc[mandals]
            max_score = score

    new_unknowns['Mandal_Nam'].iloc[line] = max_crop
    new_unknowns['Matching Dist'].iloc[line] = dist
    max_score = 0
    max_crop = ''

In [36]:
for i in range(0, len(new_unknowns)):
    if (new_unknowns['Dist_Name'].iloc[i] == 'Sangareddy') & (new_unknowns['Actual Mandal'].iloc[i] == 'Gandipet'):
        new_unknowns['Mandal_Nam'].iloc[i]='Unknown'
    elif (new_unknowns['Dist_Name'].iloc[i] == 'Bhadradri Kothagudem') & (new_unknowns['Actual Mandal'].iloc[i] == 'Onions'):
        new_unknowns['Mandal_Nam'].iloc[i]='Unknown'
    elif (new_unknowns['Dist_Name'].iloc[i] == 'Jayashankar Bhupalpally') & ((new_unknowns['Actual Mandal'].iloc[i] == 'Wajedu') | (new_unknowns['Actual Mandal'].iloc[i] == 'Mahamutharam') | (new_unknowns['Actual Mandal'].iloc[i] == 'Mahamuttaram')):
        new_unknowns['Mandal_Nam'].iloc[i]='Unknown'
    elif (new_unknowns['Dist_Name'].iloc[i] == 'Sangareddy') & ((new_unknowns['Actual Mandal'].iloc[i] == 'Velimela')):
        new_unknowns['Mandal_Nam'].iloc[i]='Unknown'
    elif (new_unknowns['Dist_Name'].iloc[i] == 'Sangareddy') & ((new_unknowns['Actual Mandal'].iloc[i] == 'Shankarampet-R')):
        new_unknowns['Mandal_Nam'].iloc[i]='Shankarampet_R'


In [37]:
new_adds2 = new_unknowns[new_unknowns['Mandal_Nam']!='Unknown']
new_adds2['Dist_Name'] = new_adds2['Matching Dist']
del new_adds2['Matching Dist']
new_unknowns2 = new_unknowns[new_unknowns['Mandal_Nam']=='Unknown']
del new_unknowns2['Matching Dist']

In [38]:
new_linked = pd.merge(new_adds2, mandal_boundaries[['Dist_Name', 'Mandal_Nam', 'index']], on =['Dist_Name', 'Mandal_Nam'], how='left')
matched_pivots2 = matched_pivots.append(new_linked)


In [39]:
#Completely Missing Mandals
not_found = pd.merge(matched_pivots2, mandal_boundaries[['Dist_Name', 'Mandal_Nam', 'index']], on =['Dist_Name', 'Mandal_Nam', 'index'], how='right')
not_found[(not_found['Year'].isnull()) & (not_found['Dist_Name']!='Hyderabad')]['index'].to_list()

[13, 34, 37, 220, 265, 579, 584, 585, 586, 587, 588, 591]

In [40]:
matched_pivots2 = matched_pivots2.groupby(['index', 'Year', 'Season', 'Mandal_Nam',
       'Dist_Name'])['Acid Lime', 'Ajwam (Vamu)', 'Amala', 'Anumulu', 'Apples',
       'Ash Gourd', 'Bajra', 'Banana', 'Batavia', 'Bengal', 'Bhendi',
       'Bitter Guard', 'Black', 'Bodakakara', 'Bottle Guard', 'Brinjal',
       'Cabbage', 'Capsicum', 'Carrot', 'Cashewnut', 'Castor', 'Cauliflower',
       'Chikkudu', 'Chillies', 'Coccinea', 'Coriander', 'Cotton', 'Cow',
       'Cucumber', 'Custard Apple', 'Danimma', 'Donda', 'Drum Stick',
       'Flowers', 'Fruit', 'Ginger', 'Gokara', 'Grapes', 'Green', 'Groundnut',
       'Guava', 'Horse', 'Horticulture', 'Jasmine', 'Jowar', 'Korra',
       'Kothemmeera', 'Maize', 'Mango', 'Mulberry', 'Mustard', 'Onions',
       'Orange', 'Other', 'Paddy', 'Palm Oil', 'Papaya', 'Peas', 'Plantain',
       'Pomegranate', 'Potato', 'Ragi', 'Red', 'Ridge Gourd', 'Rose',
       'Safflower', 'Sesamum', 'Soybean', 'Sugarcane', 'Sunflower', 'Tobacco',
       'Tomatoes', 'Total', 'Turmeric', 'Vegetables', 'Wheat'].sum().reset_index()

In [41]:
total = pd.merge(matched_pivots2, mandal_boundaries[['Dist_Name', 'Mandal_Nam', 'index']], on=['Dist_Name', 'Mandal_Nam','index'], how = 'inner')

In [42]:
total = total[['Season', 'Year', 'index' , 'Mandal_Nam', 'Dist_Name', 'Paddy', 'Maize', 'Soybean', 'Cotton', 'Chillies', 'Turmeric']]

In [43]:
total['Year'] = total['Year'].astype(int)

In [44]:
df_total = pd.merge(df, total, on=['index', 'Mandal_Nam', 'Dist_Name', 'Season', 'Year'], how='left')

In [45]:
off_season = df_total.loc[df_total.Season == 'None']
hyderabad = df_total.loc[df_total['Dist_Name']=='Hyderabad'] 

In [46]:
off_season[['Paddy', 'Maize', 'Soybean', 'Cotton', 'Chillies', 'Turmeric']] = off_season[['Paddy', 'Maize', 'Soybean', 'Cotton', 'Chillies', 'Turmeric']].fillna(value=0)
hyderabad[['Paddy', 'Maize', 'Soybean', 'Cotton', 'Chillies', 'Turmeric']] = hyderabad[['Paddy', 'Maize', 'Soybean', 'Cotton', 'Chillies', 'Turmeric']].fillna(value=0)

In [47]:
df_total.loc[df_total.Season == 'None'] = off_season
df_total.loc[df_total['Dist_Name']=='Hyderabad'] = hyderabad

In [48]:
df_total = df_total.reset_index()
del df_total['level_0'] 

In [49]:
final_selected = df_total

In [50]:
# final_lags = data[['index', 'ModifiedDateTime', 'SSM_mean', 'SSM_median', 'SSM_min', 'SSM_max', 'SSM_percentile_90', 'ST_mean', 'ST_median', 'ST_min', 'ST_max', 'ST_percentile_90', 'BA_mean', 'BA_median', 'BA_sum', 'AOT_mean', 'AOT_median', 'AOT_min', 'AOT_max', 'AOT_percentile_90', 'PRE_mean', 'PRE_median', 'PRE_min', 'PRE_max', 'PRE_percentile_90']].groupby(["index"])
# dflist = [lag_by_group(g, final_lags.get_group(g)) for g in final_lags.groups.keys()]
# lagged_12months = pd.concat(dflist, axis=0).reset_index()
# lagged_12months.columns


In [51]:
# lagged_12months.columns = ['ModifiedDateTime', 'index', 'SSM_lagged', 'SSM_lagged_median', 'ST_lagged', 'ST_max_lagged', 'ST_min_lagged', 'ST_percentile_90_lagged', 'ST_median_lagged', 'BA_lagged', 'AOT_lagged', 'AOT_max_lagged', 'AOT_min_lagged', 'AOT_percentile_90_lagged', 'AOT_median_lagged', 'PRE_lagged', 'PRE_max_lagged', 'PRE_min_lagged', 'PRE_percentile_90_lagged', 'PRE_median_lagged', 'group']
# lagged_12months = lagged_12months[lagged_12months['ModifiedDateTime']>='2016-01-01']
# lagged_12months['index'] = lagged_12months['index'].astype(int)

In [52]:
#final_selected = data[data['ModifiedDateTime']>='2016-01-01']
# final_df = pd.merge(final_selected, lagged_12months[['ModifiedDateTime', 'index', 'SSM_lagged', 'SSM_lagged_median', 'ST_lagged', 'ST_max_lagged', 'ST_min_lagged', 'ST_percentile_90_lagged', 'ST_median_lagged', 'BA_lagged', 'AOT_lagged', 'AOT_max_lagged', 'AOT_min_lagged', 'AOT_percentile_90_lagged', 'AOT_median_lagged', 'PRE_lagged', 'PRE_max_lagged', 'PRE_min_lagged', 'PRE_percentile_90_lagged', 'PRE_median_lagged']], on=['ModifiedDateTime', 'index'], how='left')

Create Soil Type dummies

In [53]:
soil_type_dummies = pd.get_dummies(final_selected['Soil Type'])

In [54]:
final_selected = pd.concat([final_selected, soil_type_dummies], axis = 1)

In [55]:
#del final_df['SSM']
#del final_df['SSM_median']
#del final_df['ST_mean']
#del final_df['ST_max']
#del final_df['ST_min']
#del final_df['ST_median']
#del final_df['ST_percentile_90']
#del final_df['AOT_mean']
#del final_df['AOT_max']
#del final_df['AOT_min']
#del final_df['AOT_median']
#del final_df['AOT_percentile_90']
#del final_df['PRE_mean']
#del final_df['PRE_max']
#del final_df['PRE_min']
#del final_df['PRE_median']
#del final_df['PRE_percentile_90']
#del final_df['BA']
#del final_df['geometry']
#del final_df['Soil Type']

In [56]:
final_selected[['index', 'Mandal_Nam', 'Dist_Name', 'geometry', 'ModifiedDateTime', 'Year', 'Month', 'Season', 'NO2', 'PM2.5', 'SSM_mean', 'SSM_median', 'SSM_min', 'SSM_max', 'SSM_percentile_90', 'PRE_mean', 'PRE_median', 'PRE_min', 'PRE_max', 'PRE_percentile_90', 'PRE_sum', 
    'POP_mean', 'POP_median', 'ST_mean', 'ST_median', 'ST_min', 'ST_max', 'ST_percentile_90', 'RWI_mean', 'RWI_median', 'RWI_min', 'RWI_max', 'RWI_percentile_90',
    'BA_mean', 'BA_median', 'BA_sum', 'Crops', 'Trees', 'Built Area', 'time_to_rur_median', 'PowP_score', 'Soil Type', 'Paddy', 'Maize', 'Soybean', 'Cotton', 'Chillies', 'Turmeric', 'Chromic Luvisols', 'Chromic Vertisols', 'Eutric Nitosols', 'Lithosols', 'Pellic Luvisols', 'Plinthic Luvisols', 'Vertic Cambisols']]

,index,Mandal_Nam,Dist_Name,geometry,ModifiedDateTime,Year,Month,Season,NO2,PM2.5,...,Cotton,Chillies,Turmeric,Chromic Luvisols,Chromic Vertisols,Eutric Nitosols,Lithosols,Pellic Luvisols,Plinthic Luvisols,Vertic Cambisols
0,0,Abdullapurmet,Rangareddy,"POLYGON ((78.59020512800009 17.34506113200013,...",2016-01-01,2016,1,None,238.435040,47.675000,...,0.0,0.0,0.0,1,0,0,0,0,0,0
1,1,Achampet,Nagarkurnool,"POLYGON ((78.91204436700008 16.54875844100007,...",2016-01-01,2016,1,None,143.419573,32.478572,...,0.0,0.0,0.0,0,0,0,1,0,0,0
2,2,Adavidevulapally,Nalgonda,"POLYGON ((79.52429751700004 16.72522135500003,...",2016-01-01,2016,1,None,289.370079,46.425000,...,0.0,0.0,0.0,0,0,0,0,1,0,0
3,3,Addagudur,Yadadri Bhuvanagiri,"POLYGON ((79.36516547400004 17.48429223000005,...",2016-01-01,2016,1,None,157.972443,42.300000,...,0.0,0.0,0.0,1,0,0,0,0,0,0
4,4,Addakal,Mahabubnagar,"POLYGON ((77.92377499000008 16.58074325200005,...",2016-01-01,2016,1,None,166.535431,40.179999,...,0.0,0.0,0.0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34923,587,Mosra,Nizamabad,"POLYGON ((78.03813400200016 18.52915054300013,...",2019-12-01,2019,12,None,210.629918,62.633331,...,0.0,0.0,0.0,1,0,0,0,0,0,0
34924,588,Chandur,Nizamabad,"POLYGON ((78.04060240200022 18.4965250740002, ...",2019-12-01,2019,12,None,200.787399,60.099998,...,0.0,0.0,0.0,1,0,0,0,0,0,0
34925,589,Chowtakur,Sangareddy,"POLYGON ((78.06919574500017 17.67324188400011,...",2019-12-01,2019,12,None,187.500000,56.275001,...,0.0,0.0,0.0,1,0,0,0,0,0,0
34926,590,Masaipet,Medak,"POLYGON ((78.44551160999998 17.94706595500003,...",2019-12-01,2019,12,None,170.078741,54.900001,...,0.0,0.0,0.0,1,0,0,0,0,0,0


In [57]:
test = final_selected.groupby('index')['Mandal_Nam'].count()

In [58]:
final_selected = final_selected.rename(columns = {'Chromic Luvisols' : 'SoilT_Chromic_Luvisols', 'Chromic Vertisols' : 'SoilT_Chromic_Vertisols' , 'Eutric Nitosols' : 'SoilT_Eutric_Nitosols', 'Lithosols' : 'SoilT_Lithosols',
       'Pellic Luvisols': 'SoilT_Pellic_luvisols', 'Plinthic Luvisols' : 'SoilT_Plinthic_Luvisols', 'Vertic Cambisols': 'SoilT_Vertic_Cambisols', 'Paddy' : 'Harvested_CropA_Paddy', 'Maize': 'Harvested_CropA_Maize', 'Soybean': 'Harvested_CropA_Soybean', 
       'Cotton': 'Harvested_CropA_Cotton', 'Chillies': 'Harvested_CropA_Chillies', 'Turmeric': 'Harvested_CropA_Turmeric', 'Crops': 'Area_Crops', 'Built Area': 'Area_Built_Area', 'Trees': 'Area_Trees'})

In [59]:
#We also want to include Amount of Fires and FRP
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Classification_Fires')
fire_data_classified = gpd.read_file('fires_data_classified.geojson')
fire_data = fire_data_classified[fire_data_classified['agricultural'] == 1]

#Create geodataframe from the data
geo_fire_data = gpd.GeoDataFrame(fire_data,geometry = fire_data.geometry, crs = {'init': 'epsg:4326'}) 

#Make sure the geometry columns are in the right format
geo_fire_data = geo_fire_data[['geometry', 'acq_date', 'fireID', 'frp']]
geo_fire_data['geometry'] = geo_fire_data['geometry'].to_crs(epsg = 4326)
boundaries = gpd.GeoDataFrame(mandal_boundaries, geometry = mandal_boundaries.geometry, crs = {'init': 'epsg:4326'}) 

#Load date into date format
geo_fire_data['acq_date'] =  pd.to_datetime(geo_fire_data['acq_date'])
geo_fire_data['year'] = (geo_fire_data['acq_date']).dt.year
geo_fire_data['month'] = (geo_fire_data['acq_date']).dt.month
geo_fire_data['day'] = (geo_fire_data['acq_date']).dt.day

#Selects the years we are interested in, depending on the input of the function
geo_fire_data = geo_fire_data[(geo_fire_data['acq_date'] >= '2016-01-01') & (geo_fire_data['acq_date'] < '2020-01-01')]

#Count all fires within a region given by the boundaries dataframe
fires_per_boundaries= gpd.sjoin(geo_fire_data, boundaries, how="inner")

#Create the right time format: We count per month
fires_per_boundaries['day'] = 1 
fires_per_boundaries['year'] = pd.Series(pd.to_numeric(fires_per_boundaries['year'], errors='coerce'), dtype='int64')
fires_per_boundaries['month'] = pd.Series(pd.to_numeric(fires_per_boundaries['month'], errors='coerce'), dtype='int64')
fires_per_boundaries['ModifiedDateTime'] = pd.to_datetime(fires_per_boundaries[['year', 'month', 'day']].astype('int64').astype('str'), yearfirst=True)

unit_number = 'Number of Fires'

#Sum amount of fires per mandal per month per year make sure that if no fire happens at a specific time write a zero
fires_per_boundaries_count = fires_per_boundaries.groupby(['index', 'ModifiedDateTime'])['fireID'].count().unstack(fill_value=0).stack().reset_index()
fires_per_boundaries_count[unit_number] = fires_per_boundaries_count[0] 
del fires_per_boundaries_count[0] 

unit_frp = 'Fire Radiative Power (MW)'
fires_per_boundaries_count_2 = fires_per_boundaries.groupby(['index', 'ModifiedDateTime'])['frp'].mean().unstack(fill_value=0).stack().reset_index()
fires_per_boundaries_count[unit_frp] = fires_per_boundaries_count_2[0] 

fires_per_boundaries_count['ModifiedDateTime'] =  pd.to_datetime(fires_per_boundaries_count['ModifiedDateTime'])

In [60]:
final_selected['ModifiedDateTime'] = pd.to_datetime(final_selected['ModifiedDateTime'])

In [61]:
final_selected = pd.merge(fires_per_boundaries_count, final_selected, on=['index', 'ModifiedDateTime'], how='right')
final_selected = final_selected.rename(columns = {'Number of Fires': 'Fires', 'Fire Radiative Power (MW)': 'FRP'}) 

In [62]:
final_selected = final_selected[(final_selected['ModifiedDateTime']>='2016-01-01') & (final_selected['ModifiedDateTime']<='2019-12-01')]

In [63]:
final_selected['Fires'] = final_selected['Fires'].fillna(0)
final_selected['FRP'] = final_selected['FRP'].fillna(0)

In [64]:
final_selected.columns

Index(['index', 'ModifiedDateTime', 'Fires', 'FRP', 'Mandal_Nam', 'Dist_Name',
       'geometry', 'Year', 'Month', 'Season', 'NO2', 'PM2.5', 'SSM_mean',
       'SSM_median', 'SSM_min', 'SSM_max', 'SSM_percentile_90', 'PRE_mean',
       'PRE_median', 'PRE_min', 'PRE_max', 'PRE_percentile_90', 'PRE_sum',
       'ST_mean', 'ST_median', 'ST_min', 'ST_max', 'ST_percentile_90',
       'RWI_mean', 'RWI_median', 'RWI_min', 'RWI_max', 'RWI_percentile_90',
       'BA_mean', 'BA_median', 'BA_sum', 'Area_Crops', 'Crops_Prop',
       'Area_Trees', 'Trees_Prop', 'Area_Built_Area', 'Built_Area_Prop',
       'time_to_rur_median', 'PowP_score', 'Soil Type', 'Area', 'POP_mean',
       'POP_median', 'Harvested_CropA_Paddy', 'Harvested_CropA_Maize',
       'Harvested_CropA_Soybean', 'Harvested_CropA_Cotton',
       'Harvested_CropA_Chillies', 'Harvested_CropA_Turmeric',
       'SoilT_Chromic_Luvisols', 'SoilT_Chromic_Vertisols',
       'SoilT_Eutric_Nitosols', 'SoilT_Lithosols', 'SoilT_Pellic_luvisols',
 

In [65]:
final_selected.head()

,index,ModifiedDateTime,Fires,FRP,Mandal_Nam,Dist_Name,geometry,Year,Month,Season,...,Harvested_CropA_Cotton,Harvested_CropA_Chillies,Harvested_CropA_Turmeric,SoilT_Chromic_Luvisols,SoilT_Chromic_Vertisols,SoilT_Eutric_Nitosols,SoilT_Lithosols,SoilT_Pellic_luvisols,SoilT_Plinthic_Luvisols,SoilT_Vertic_Cambisols
0,0,2016-01-01,0.0,0.0,Abdullapurmet,Rangareddy,"POLYGON ((78.59020512800009 17.34506113200013,...",2016,1,None,...,0.0,0.0,0.0,1,0,0,0,0,0,0
1,1,2016-01-01,0.0,0.0,Achampet,Nagarkurnool,"POLYGON ((78.91204436700008 16.54875844100007,...",2016,1,None,...,0.0,0.0,0.0,0,0,0,1,0,0,0
2,2,2016-01-01,0.0,0.0,Adavidevulapally,Nalgonda,"POLYGON ((79.52429751700004 16.72522135500003,...",2016,1,None,...,0.0,0.0,0.0,0,0,0,0,1,0,0
3,3,2016-01-01,0.0,0.0,Addagudur,Yadadri Bhuvanagiri,"POLYGON ((79.36516547400004 17.48429223000005,...",2016,1,None,...,0.0,0.0,0.0,1,0,0,0,0,0,0
4,4,2016-01-01,0.0,0.0,Addakal,Mahabubnagar,"POLYGON ((77.92377499000008 16.58074325200005,...",2016,1,None,...,0.0,0.0,0.0,0,0,0,0,1,0,0


In [66]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\sandbox\\notebooks\\crop_fires\\Model\\Data')
final_selected.to_csv('Data_combined.csv')